In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of calc failed: Traceback (most recent call last):
  File "/home/eric/anaconda3/envs/refi/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/eric/anaconda3/envs/refi/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/eric/anaconda3/envs/refi/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/eric/anaconda3/envs/refi/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 779, in exec_module
  File "<frozen importlib._bootstrap_external>", line 916, in get_code
  File "<frozen importlib._bootstrap_external>", line 846, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File

In [88]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_table
from dash.dependencies import Input, Output
from datetime import datetime
import pandas as pd
import numpy as np
import numpy_financial as npf
import plotly.express as px
# from dash_table import DataTable, FormatTemplate

from calc import *

In [48]:
def create_mortage_range(principal, term, rmax=0.05, rstep=0.0125):
    df = pd.DataFrame(data={'rate':np.arange(0,rmax+rstep,rstep)})
    df['monthly_payment'] = df.apply(lambda x: calc_loan_monthly_payment(principal, x['rate'], term), axis=1)
    df['total_payment'] = df.apply(lambda x: total_payment(x['monthly_payment'], term), axis=1)
    print(df)
    return df


def create_value_plot(principal, term, 
                      x='rate', 
                      y='monthly_payment', 
                      title='Monthly Payment by Interest Rate'):
    df = create_mortage_range(principal, term)
    fig = px.line(df, x=x, y=y, title=title)
    
    #current values
    fig.add_shape(type='line',
                x0=0,
                y0=2015.11,
                x1=0.0250,
                y1=2015.11,
                line=dict(color='Black',),
                xref='x',
                yref='y'
    )
    
    fig.add_shape(type='line',
                x0=0.0250,
                y0=1000,
                x1=0.0250,
                y1=2015.11,
                line=dict(color='Black',),
                xref='x',
                yref='y'
    )
    
    #target values
    fig.add_shape(type='line',
                x0=0,
                y0=1699.58,
                x1=0.0125,
                y1=1699.58,
                line=dict(color='Red',),
                xref='x',
                yref='y'
    )
    
    fig.add_shape(type='line',
                x0=0.0125,
                y0=1000,
                x1=0.0125,
                y1=1699.58,
                line=dict(color='Red',),
                xref='x',
                yref='y'
    )

    return fig


In [49]:
f = create_value_plot(510000, 360)
f.show()

     rate  monthly_payment  total_payment
0  0.0000      1416.666667  510000.000000
1  0.0125      1699.583588  611850.091682
2  0.0250      2015.116584  725441.970229
3  0.0375      2361.889517  850280.226126
4  0.0500      2737.790277  985604.499850


In [50]:
find_target_interest_rate(510000, 360, 1600)

0.0075

In [51]:
def create_staged_value_plot(principal, remaining_principal, term, x='rate', y='monthly_payment', title='Monthly Payment by Interest Rate'):
    df = create_mortage_range(principal, term)
    dfc = create_mortage_range(remaining_principal, term)

    fig = px.line(df, x=x, y=y, title=title)
    fig.append_trace({'x':dfc[x],'y':dfc[y],'type':'scatter','name':'Remaining Principal'},1,1)
    
    fig.add_shape(type='line',
                x0=0,
                y0=2015.11,
                x1=0.0250,
                y1=2015.11,
                line=dict(color='Black',),
                xref='x',
                yref='y'
    )
    
    fig.add_shape(type='line',
                x0=0.0250,
                y0=1000,
                x1=0.0250,
                y1=2015.11,
                line=dict(color='Black',),
                xref='x',
                yref='y'
    )
    
    #target values
    fig.add_shape(type='line',
                x0=0,
                y0=1699.58,
                x1=0.0125,
                y1=1699.58,
                line=dict(color='Red',),
                xref='x',
                yref='y'
    )
    
    fig.add_shape(type='line',
                x0=0.0125,
                y0=1000,
                x1=0.0125,
                y1=1699.58,
                line=dict(color='Red',),
                xref='x',
                yref='y'
    )
    
    return fig


In [52]:
create_staged_value_plot(510000,490000,360)

     rate  monthly_payment  total_payment
0  0.0000      1416.666667  510000.000000
1  0.0125      1699.583588  611850.091682
2  0.0250      2015.116584  725441.970229
3  0.0375      2361.889517  850280.226126
4  0.0500      2737.790277  985604.499850
     rate  monthly_payment  total_payment
0  0.0000      1361.111111  490000.000000
1  0.0125      1632.933251  587855.970440
2  0.0250      1936.092404  696993.265514
3  0.0375      2269.266399  816935.903533
4  0.0500      2630.425953  946953.342993


In [53]:
def amount_remaining(principal, monthly_payment, rate, months_remaining):
    return (monthly_payment/(rate/12)) * (1 - (1/(1+(rate/12))**months_remaining))

def create_mortgage_table(principal, rate, term):
    '''
    B = (PMT/R) x (1 - (1/(1+R)^N) In the formula, "B" is the principal balance, 
    "PMT" is the monthly payment for principal and interest and "N" is the number of months remaining.
    "R" is your interest rate, but it's expressed as a monthly rate rather than an annual one. 
    To get the monthly rate, simply take your annual rate, expressed as a decimal, and divide it by 12. 
    If your annual rate is 6 percent, for example: R = 0.06/12 = 0.005.
    '''
    df = pd.DataFrame(data={'month':np.arange(0,term+1,1)})
    monthly_payment = calc_loan_monthly_payment(principal, rate, term)
    
    df['months_remaining'] = term - df['month']                                     
    df['amount_remaining'] = df['months_remaining'].map(lambda x: amount_remaining(principal, monthly_payment, rate, x))
    return df


def understand_mortgage_extension(original_principal, 
                                  remaining_principal, 
                                  original_rate, 
                                  new_rate,
                                  original_term,
                                  remaining_term, 
                                  new_term):
    df_original = create_mortgage_table(original_principal, original_rate, original_term)
    df_refi = create_mortgage_table(remaining_principal, new_rate, new_term)
    df_refi['month'] = df_refi['month'] + remaining_term
    
    df_original_pre = df_original[df_original['month']<=remaining_term]
    df_original_post = df_original[df_original['month']>remaining_term]
    
    fig = px.line(df_original_pre, x='month', y='amount_remaining')
    fig.append_trace({'x':df_refi['month'],
                      'y':df_refi['amount_remaining'],
                      'type':'scatter',
                      'line':{'dash': 'solid', 'color': 'green'},
                      'name':'Current Refinance Scenario'},1,1)
    fig.append_trace({'x':df_original_post['month'],
                      'y':df_original_post['amount_remaining'],
                      'type':'scatter',
                      'line':{'dash': 'dash', 'color': 'blue'},
                      'name':'Original Loan'},1,1)
    
    
    return fig
    

In [54]:
understand_mortgage_extension(510000,306038, 0.0425, 0.02875, 360, 200, 360)

In [62]:
def create_efficient_frontier(original_principal,
                              original_rate,
                              original_term,
                              current_principal,
                              term_remaining,
                              new_term,
                              refi_cost):
    original_monthly_payment = calc_loan_monthly_payment(original_principal, original_rate, original_term)
    original_total_payment = total_payment(original_monthly_payment, original_term)
    new_target_total = original_total_payment - refi_cost - (original_term - term_remaining)*original_monthly_payment
    print("original_total", original_total_payment)
    print("paid_to_date", (original_term - term_remaining)*original_monthly_payment)
    print("new_target", new_target_total)
    
#     df = create_mortgage_table(original_principal, original_rate, original_term)
#     df['interest_rate'] = df['amount_remaining'].map(lambda x: find_break_even_interest(x, new_term, new_target_total, original_rate))
    
    return df
    

In [63]:
create_efficient_frontier(510000, 0.02875,360,220446.5,240,360,10000)

original_total 761742.5879117125
paid_to_date 253914.1959705708
new_target 497828.3919411417


NameError: name 'df' is not defined

In [87]:
find_break_even_interest(220446.5, 360, 300000, 0.02875)

0.021249999999999995

In [77]:
original_monthly_payment = calc_loan_monthly_payment(558000, 0.0425, 360)
original_total_payment = total_payment(original_monthly_payment, 360)
new_target_total = original_total_payment - 10000 - (360 - 240)*original_monthly_payment
total_at_refi = total_payment(calc_loan_monthly_payment(220446.5, 0.0425, 360),360)

In [78]:
new_target_total

648805.9021336343

In [79]:
total_at_refi

390407.1379158662

In [80]:
original_total_payment

988208.8532004516

In [81]:
original_total_payment - ((360 - 240)*original_monthly_payment + 10000 + total_at_refi)

258398.7642177682

In [82]:
original_total_payment

988208.8532004516

In [91]:
principal = 510000
rate = 0.02875
term = 360
per = np.arange(term) + 1

In [93]:
npf.ipmt(rate/12, per,1*12, principal)

array([-1221.875     , -1121.38685594, -1020.65795903,  -919.68773248,
        -818.47559809,  -717.0209763 ,  -615.32328614,  -513.38194526,
        -411.19636993,  -308.76597498,  -206.09017388,  -103.16837867,
          -0.        ,   103.41555291,   207.07887226,   310.99055164,
         415.15118608,   519.56137205,   624.22170742,   729.1327915 ,
         834.29522506,   939.70961029,  1045.37655081,  1151.29665171,
        1257.47051952,  1363.89876222,  1470.58198926,  1577.52081152,
        1684.71584138,  1792.16769266,  1899.87698068,  2007.84432219,
        2116.07033546,  2224.55564022,  2333.30085769,  2442.30661057,
        2551.57352308,  2661.10222089,  2770.89333121,  2880.94748273,
        2991.26530566,  3101.8474317 ,  3212.69449408,  3323.80712756,
        3435.18596838,  3546.83165435,  3658.74482477,  3770.92612049,
        3883.3761839 ,  3996.09565892,  4109.08519102,  4222.34542721,
        4335.87701604,  4449.68060764,  4563.75685368,  4678.10640739,
      